In [386]:
import matplotlib.pyplot as plt
import numpy as np
import pulp 
import pandas as pd


In [387]:
#Read data 
track_data = pd.read_excel("../dataset/ariane_tracks_booster_container_1.0.xlsx")
planning_data = pd.read_excel("../dataset/a6_booster_delivery_planning.xlsx")

In [388]:
# Create a LP minimization problem
prob = pulp.LpProblem("Logistics_Optimization", pulp.LpMinimize)

# Define decision variables: X[i][j] represents the amount of goods shipped from warehouse i to customer j
# Decision variable is continuous and non-negative
locations = ['A', 'B', 'C','D','E','D','C','B']
containers = ['SN7', 'SN3', 'SN8', 'SN6', 'SN2', 'SN1', 'SN4', 'SN5']
dates = np.arange(0,91)

distance_travelled_truck  = 400 #600 to 800 km
truck_co2_emission = 0.105*10**(-3)*distance_travelled_truck #tones
conopee_co2_emission = 16800 # tones todo

num_containers=8
capacity_ship=4

# Define costs of transportation from each warehouse to each customer
co2_emissions = {
    ('A', 'B'): truck_co2_emission,
    ('B', 'C'): truck_co2_emission,
    ('C', 'D'): conopee_co2_emission,
    ('D', 'E'): truck_co2_emission
}

duration_trip =  {
    ('A', 'B'): 1,
    ('B', 'C'): 1,
    ('C', 'D'): 10,
    ('D', 'E'): 1,
    ('E', 'D'): 1,
    ('D', 'C'): 10,
    ('C', 'B'): 1,
    ('B', 'A'): 1
}

nominal_lead_time = {
    'A':15,
    'B':5,
    'C':1,
    'D':2,
    'E':15
}

In [389]:
current_date='01.01.2024'
current_date=pd.to_datetime(current_date).normalize()
future_date = current_date + pd.DateOffset(months=3)
# Filter the DataFrame
config_data = planning_data[(planning_data["Date"]>= current_date) & (planning_data["Date"] <= future_date)]

#Get the configuration
start_date = current_date
end_date = future_date
dates_tmp= pd.date_range(start=start_date, end=end_date, freq='D')

# Format the dates
formatted_dates = dates_tmp.strftime('%Y-%m-%d')
df = pd.DataFrame({'Date': formatted_dates})
df["Date"]=pd.to_datetime(df["Date"])

merged_df = pd.concat([df.set_index('Date'), config_data.set_index('Date')], axis=1)

merged_df['Config'].fillna(value=0, inplace=True)
merged_df.reset_index(inplace=True)

config = merged_df['Config'].values
print(config)

#Get config data in right format
dates_config = np.where([config>0])[1]

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 4. 0. 0. 0. 0. 0. 0. 2. 0. 0.]


/tmp/ipykernel_120788/3946278762.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['Config'].fillna(value=0, inplace=True)


In [390]:
index = []
for i,location_start in enumerate(locations):
     location_end = locations[(i+1)%len(locations)]
     for date in dates:
          #date when containers leave location_start
          index.append((date,location_start,location_end))
          index.append((date,location_end,location_start))
"""
index = []
location_start='C'
location_end='D'
for date in dates:
     #date when containers leave location_start
     index.append((date,location_start,location_end))
     index.append((date,location_end,location_start))
"""

"\nindex = []\nlocation_start='C'\nlocation_end='D'\nfor date in dates:\n     #date when containers leave location_start\n     index.append((date,location_start,location_end))\n     index.append((date,location_end,location_start))\n"

In [403]:
# Define the variables
nr_containers = pulp.LpVariable.dicts("n",index, lowBound=0,upBound=capacity_ship, cat="Integer")
storage_containers = pulp.LpVariable.dicts("s",[(date,location) for date in dates for location in locations], lowBound=0,upBound=capacity_ship, cat="Integer")
lateness_slack = pulp.LpVariable.dicts("a",dates_config, lowBound=0, cat="Continuous")
earliness_slack = pulp.LpVariable.dicts("b",dates_config, lowBound=0, cat="Continuous")
ship = pulp.LpVariable.dicts("y",[(loc,date) for loc in ['C','CD','DC','D'] for date in dates], cat="Binary")

In [405]:
# Objective function: minimize total transportation cost
prob = pulp.LpProblem("Logistics_Optimization", pulp.LpMinimize)
alpha_late = 1.0 #TODO: probably increase
alpha_early =0.5
alpha_emission = 1.0
# Define the objective function
prob += (alpha_late * pulp.lpSum(lateness_slack[date] for date in dates_config) +
         alpha_early * pulp.lpSum(earliness_slack[date] for date in dates_config) +
         alpha_emission * (pulp.lpSum((5 - nr_containers[date, 'C', 'D']) for date in dates) +
                           pulp.lpSum((5 - nr_containers[date, 'D', 'C']) for date in dates)))

In [407]:
nr_containers[(date,'C','D')]

n_(0,_'C',_'D')

In [409]:
ship_locations= ['C','CD','DC','D'] 
#sum up to 1
for date in dates:  
    #sum op to 1
    prob += pulp.lpSum(ship[(loc,date)]for loc in ship_locations )==1
    #ship iff only if ship 1->0
    prob+= 4*(1-ship[('C',date+1)]) >= nr_containers[(date,'C','D')]
    prob+= ship[('C',date)] - ship[('C',date+1)] <= nr_containers[(date,'C','D')]

    prob+=ship[('C',date)]>=ship[('CD',date+1)]
    prob+=ship[('CD',date+1)]>=ship[('C',date)] - ship[('C',date+1)]

    for t in range(1,duration_trip[('C','D')]):
        print(t)
        prob+=ship[('CD',date+1)]==ship[('CD',date+t)]
    
    prob+=ship[('D',date+1)]==ship[('CD',date+t)]



In [393]:
# On time delivery
previous_date = 0
for date in dates_config:
    next_date = date
    print(previous_date,next_date)
    print(config[next_date] + lateness_slack[next_date])
    print(config[next_date] - earliness_slack[next_date])
    #Time spend transporting container and staying at location
    time_between=duration_trip[('C','D')] +duration_trip[('D','E')]  + nominal_lead_time['D']
    #Get the number of containers delivered in time to fulfill the demand
    prob += storage_containers[next_date,'E']<=config[next_date] + lateness_slack[next_date] # ? is next_date in storage_containers correct
    prob += storage_containers[next_date,'E']>= config[next_date] - earliness_slack[next_date]
    previous_date = next_date 

0 15
a_15 + 2.0
-b_15 + 2.0
15 82
a_82 + 4.0
-b_82 + 4.0
82 89
a_89 + 2.0
-b_89 + 2.0


In [394]:
#Update storage:
for i,location in enumerate(locations):
    for date in dates:
        if date+max(duration_trip.values())<=dates[-1]:
            next_location= locations[(i+1)%len(locations)]
            #Take number of containers out of the storage
            prob+=storage_containers[date+1,location]==storage_containers[date,location]-nr_containers[date,location,next_location]
            #We can't transport more then is in storage
            prob+=nr_containers[date,location,next_location] <= storage_containers[date,location]  
            #Containers arrive in the next location
            prob+=storage_containers[date+duration_trip[(location,next_location)],next_location] == storage_containers[date+duration_trip[(location,next_location)]-1,next_location] + nr_containers[date,location,next_location]              

In [396]:
# Number of containers can not be exceeded
for date in dates:
    #for i,location in enumerate(locations[:4]):
        #print(date, location, locations[(i+1)%len(locations)])
        #print(storage_containers[date,location] + nr_containers[date, location, locations[(i+1)%len(locations)]] +nr_containers[date, locations[(i+1)%len(locations)], location])
    #print("---")    
    prob+=pulp.lpSum(storage_containers[date,location] + nr_containers[date, location, locations[(i+1)%len(locations)]] 
                     +nr_containers[date, locations[(i+1)%len(locations)], location]
                     for i,location in enumerate(locations[:4])) + storage_containers[date,'E']==num_containers

In [ ]:
#one boat
for date in dates:
    nr_containers[date,'C','D']

In [397]:
prob

Logistics_Optimization:
MINIMIZE
1.0*a_15 + 1.0*a_82 + 1.0*a_89 + 0.5*b_15 + 0.5*b_82 + 0.5*b_89 + -1.0*n_(0,_'C',_'D') + -1.0*n_(0,_'D',_'C') + -1.0*n_(1,_'C',_'D') + -1.0*n_(1,_'D',_'C') + -1.0*n_(10,_'C',_'D') + -1.0*n_(10,_'D',_'C') + -1.0*n_(11,_'C',_'D') + -1.0*n_(11,_'D',_'C') + -1.0*n_(12,_'C',_'D') + -1.0*n_(12,_'D',_'C') + -1.0*n_(13,_'C',_'D') + -1.0*n_(13,_'D',_'C') + -1.0*n_(14,_'C',_'D') + -1.0*n_(14,_'D',_'C') + -1.0*n_(15,_'C',_'D') + -1.0*n_(15,_'D',_'C') + -1.0*n_(16,_'C',_'D') + -1.0*n_(16,_'D',_'C') + -1.0*n_(17,_'C',_'D') + -1.0*n_(17,_'D',_'C') + -1.0*n_(18,_'C',_'D') + -1.0*n_(18,_'D',_'C') + -1.0*n_(19,_'C',_'D') + -1.0*n_(19,_'D',_'C') + -1.0*n_(2,_'C',_'D') + -1.0*n_(2,_'D',_'C') + -1.0*n_(20,_'C',_'D') + -1.0*n_(20,_'D',_'C') + -1.0*n_(21,_'C',_'D') + -1.0*n_(21,_'D',_'C') + -1.0*n_(22,_'C',_'D') + -1.0*n_(22,_'D',_'C') + -1.0*n_(23,_'C',_'D') + -1.0*n_(23,_'D',_'C') + -1.0*n_(24,_'C',_'D') + -1.0*n_(24,_'D',_'C') + -1.0*n_(25,_'C',_'D') + -1.0*n_(25,_'D',_'C

In [398]:
# Initialies storage in day 0:
#Take number of containers oout of the storage
prob+=storage_containers[0,'A']==4
prob+=storage_containers[0,'B']==2
prob+=storage_containers[0,'C']==2
prob+=storage_containers[0,'D']==0
prob+=storage_containers[0,'E']==0

In [399]:
#solver = pulp.getSolver('GUROBI')
#prob.solve(solver=solver)
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/hillary/miniconda3/envs/green/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/2d146b6bba484c008c8ae8ea6ebbf824-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/2d146b6bba484c008c8ae8ea6ebbf824-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2051 COLUMNS
At line 10990 RHS
At line 13037 BOUNDS
At line 14221 ENDATA
Problem MODEL has 2046 rows, 1189 columns and 6384 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -50 - 0.01 seconds
Cgl0004I processed model has 5 rows, 7 columns (4 integer (0 of which binary)) and 10 elements
Cutoff increment increased from 1e-05 to 0.4999
Cbc0012I Integer solution of -50 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
Cbc0001I Search completed - best objective -50, took 0 iterations and 0 node

1

In [400]:
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost =", pulp.value(prob.objective))


a_15 = 0.0
a_82 = 0.0
a_89 = 0.0
b_15 = 2.0
b_82 = 4.0
b_89 = 2.0
n_(0,_'A',_'B') = 0.0
n_(0,_'B',_'A') = 0.0
n_(0,_'B',_'C') = 0.0
n_(0,_'C',_'B') = 0.0
n_(0,_'C',_'D') = 0.0
n_(0,_'D',_'C') = 0.0
n_(0,_'D',_'E') = 0.0
n_(0,_'E',_'D') = 0.0
n_(1,_'A',_'B') = 0.0
n_(1,_'B',_'A') = 0.0
n_(1,_'B',_'C') = 0.0
n_(1,_'C',_'B') = 0.0
n_(1,_'C',_'D') = 0.0
n_(1,_'D',_'C') = 0.0
n_(1,_'D',_'E') = 0.0
n_(1,_'E',_'D') = 0.0
n_(10,_'A',_'B') = 0.0
n_(10,_'B',_'A') = 0.0
n_(10,_'B',_'C') = 0.0
n_(10,_'C',_'B') = 0.0
n_(10,_'C',_'D') = 0.0
n_(10,_'D',_'C') = 0.0
n_(10,_'D',_'E') = 0.0
n_(10,_'E',_'D') = 0.0
n_(11,_'A',_'B') = 0.0
n_(11,_'B',_'A') = 0.0
n_(11,_'B',_'C') = 0.0
n_(11,_'C',_'B') = 0.0
n_(11,_'C',_'D') = 0.0
n_(11,_'D',_'C') = 0.0
n_(11,_'D',_'E') = 0.0
n_(11,_'E',_'D') = 0.0
n_(12,_'A',_'B') = 0.0
n_(12,_'B',_'A') = 0.0
n_(12,_'B',_'C') = 0.0
n_(12,_'C',_'B') = 0.0
n_(12,_'C',_'D') = 0.0
n_(12,_'D',_'C') = 0.0
n_(12,_'D',_'E') = 0.0
n_(12,_'E',_'D') = 0.0
n_(13,_'A',_'B') = 0.0
n_(13,_